In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

#### 0 - Read prepated somatic exome
- RNA Vafs

In [ ]:
somatic <- 
fread(paste0(TMP_DIR, "somatic_exome.csv")) %>% 
 mu(pos10 = round(position/10), pos30 = round(position/30))

#### 1 - Get Hotspots

- Only keep hotspot if available in 1/100 over entire database

In [ ]:
threshold <- (1/100) * length(unique(somatic %>% pu(sampleId)))

In [ ]:
hot <- somatic %>% gb(gene, chromosome, position) %>% su(ct = n()) %>% fi( ct > threshold ) %>% ug()
hot10 <- somatic %>% gb(gene, chromosome, pos10) %>% su(ct = n()) %>% fi( ct > threshold ) %>% ug()
hot30 <- somatic %>% gb(gene, chromosome, pos30) %>% su(ct = n()) %>% fi( ct > threshold ) %>% ug()

- Function to prepare hotspots as biomarker table

In [ ]:
maker <- function( hotspot_df, column = "position") {
 s1 <- somatic %>% 
  ij( hotspot_df %>% se(gene, {{column}}), by = c("gene", {{column}})) %>% 
  se( sampleId, gene, {{column}} ) 
    
 if(column == "position") { s2 <- s1 %>% mu( hotspot = paste0( "hotspot_", gene, "_", column, "_", position ), ct = 1) } 
 else if (column == "pos10") {s2 <- s1 %>% mu( hotspot = paste0( "hotspot_", gene, "_", column, "_", pos10), ct = 1) } 
 else { s2 <- s1 %>% mu( hotspot = paste0( "hotspot_", gene, "_", column, "_", pos30), ct = 1)  }
    
 s2 %>% 
  se(-gene, -{{column}} ) %>% 
  unique() %>% 
  sp(hotspot, ct)
}

In [ ]:
hotspots_ready <- 
maker(hot, column = "position") %>% 
 lj(maker(hot10, column = "pos10"), by = "sampleId") %>% 
 lj(maker(hot30, column = "pos30"), by = "sampleId")

### 2 - Send it! 

In [ ]:
fwrite(hotspots_ready, paste0(READY_DIR, "hotspots_ready.csv"))